#📌 Extracción

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json

In [2]:
def extraer_datos_api():
    url = "https://raw.githubusercontent.com/alura-cursos/challenge2-data-science-LATAM/main/TelecomX_Data.json"
    
    try:
        print("📡 Extrayendo datos desde la API...")
        response = requests.get(url)
        response.raise_for_status()
        
        # Cargar JSON
        data = response.json()
        print(f"✅ Datos extraídos exitosamente: {len(data)} registros")
        
        return data
    
    except requests.exceptions.RequestException as e:
        print(f"Error al obtener datos de la API: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"Error al decodificar JSON: {e}")
        return None

# Extraer datos
raw_data = extraer_datos_api()


📡 Extrayendo datos desde la API...
✅ Datos extraídos exitosamente: 7267 registros


#🔧 Transformación

In [3]:
if raw_data:
    df = pd.json_normalize(raw_data)
    print("📊 DataFrame creado con éxito")
    print(df.head())

# Información básica del dataset
print("📋 Información básica del dataset:")
print(f"• Dimensiones: {df.shape}")
print(f"• Número de clientes: {df.shape[0]:,}")
print(f"• Número de variables: {df.shape[1]}")

# Mostrar primeras filas
print("\n🔍 Primeras 5 filas del dataset:")
print(df.head())

# Información de tipos de datos
print("\n📊 Tipos de datos:")
print(df.dtypes)

📊 DataFrame creado con éxito
   customerID Churn customer.gender  customer.SeniorCitizen customer.Partner  \
0  0002-ORFBO    No          Female                       0              Yes   
1  0003-MKNFE    No            Male                       0               No   
2  0004-TLHLJ   Yes            Male                       0               No   
3  0011-IGKFF   Yes            Male                       1              Yes   
4  0013-EXCHZ   Yes          Female                       1              Yes   

  customer.Dependents  customer.tenure phone.PhoneService phone.MultipleLines  \
0                 Yes                9                Yes                  No   
1                  No                9                Yes                 Yes   
2                  No                4                Yes                  No   
3                  No               13                Yes                  No   
4                  No                3                Yes                  No   

  i

In [13]:
def limpiar_datos(df: pd.DataFrame) -> pd.DataFrame:
    df_clean = df.copy()
    print("🔧 Iniciando limpieza de datos...")

    # 1) Valores nulos
    print("• Valores nulos por columna:")
    null_counts = df_clean.isnull().sum()
    if null_counts.sum() > 0:
        print(null_counts[null_counts > 0])
    else:
        print("  ✅ No se encontraron valores nulos")

    # 2) Duplicados
    duplicados = df_clean.duplicated().sum()
    print(f"• Registros duplicados: {duplicados}")
    if duplicados > 0:
        df_clean = df_clean.drop_duplicates()
        print(f"  ✅ {duplicados} registros duplicados eliminados")

    # 3) Conversión de tipos de datos
    print("• Convirtiendo tipos de datos...")

    # Helper: mapear Yes/No/Male/Female a 1/0 (robusto con lower())
    def map_bin(s):
        return s.astype(str).str.strip().str.lower().map({
            'yes': 1, 'no': 0,
            'male': 1, 'female': 0,
            'no internet service': 0,
            'no phone service': 0
        })

    # Manteniendo nombres originales del dataset
    binary_cols = [
        'customer.gender',
        'customer.Partner',
        'customer.Dependents',
        'phone.PhoneService',
        'account.PaperlessBilling',
        'Churn'
    ]
    for col in binary_cols:
        if col in df_clean.columns:
            df_clean[col] = map_bin(df_clean[col]).astype('Int64')

    # 4) Numéricos (tenure y cargos)
    numeric_cols = [
        'customer.tenure',
        'account.Charges.Monthly',
        'account.Charges.Total'
    ]
    for col in numeric_cols:
        if col in df_clean.columns:
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

    # 5) Categóricas (solo reporte de consistencia, no se renombran ni codifican)
    categorical_cols = [
        'phone.MultipleLines',
        'internet.InternetService',
        'internet.OnlineSecurity',
        'internet.OnlineBackup',
        'internet.DeviceProtection',
        'internet.TechSupport',
        'internet.StreamingTV',
        'internet.StreamingMovies',
        'account.Contract',
        'account.PaymentMethod'
    ]
    print("• Verificando consistencia en variables categóricas...")
    for col in categorical_cols:
        if col in df_clean.columns:
            print(f"  - {col}: {df_clean[col].nunique()} valores únicos")

    print("✅ Limpieza de datos completada")
    return df_clean


# Ejemplo de uso
df_clean = limpiar_datos(df)


🔧 Iniciando limpieza de datos...
• Valores nulos por columna:
  ✅ No se encontraron valores nulos
• Registros duplicados: 0
• Convirtiendo tipos de datos...
• Verificando consistencia en variables categóricas...
  - phone.MultipleLines: 3 valores únicos
  - internet.InternetService: 3 valores únicos
  - internet.OnlineSecurity: 3 valores únicos
  - internet.OnlineBackup: 3 valores únicos
  - internet.DeviceProtection: 3 valores únicos
  - internet.TechSupport: 3 valores únicos
  - internet.StreamingTV: 3 valores únicos
  - internet.StreamingMovies: 3 valores únicos
  - account.Contract: 3 valores únicos
  - account.PaymentMethod: 4 valores únicos
✅ Limpieza de datos completada


In [14]:
df_clean.head()  # Mostrar las primeras filas del DataFrame limpio

,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
0,0002-ORFBO,0,0,0,1,1,9,1,No,DSL,...,Yes,No,Yes,Yes,No,One year,1,Mailed check,65.6,593.30
1,0003-MKNFE,0,1,0,0,0,9,1,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,0,Mailed check,59.9,542.40
2,0004-TLHLJ,1,1,0,0,0,4,1,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,1,Electronic check,73.9,280.85
3,0011-IGKFF,1,1,1,1,0,13,1,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,1,Electronic check,98.0,1237.85
4,0013-EXCHZ,1,0,1,1,0,3,1,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,1,Mailed check,83.9,267.40


#📊 Carga y análisis

In [20]:
def analisis_churn_basico(df: pd.DataFrame):
    """
    Análisis básico de la variable objetivo (Churn).
    Requiere que Churn esté codificado como 0 = No, 1 = Sí.
    """
    print("📊 ANÁLISIS DE CHURN (Variable Objetivo)")
    print("-" * 40)

    # Distribución de Churn
    churn_counts = df['Churn'].value_counts().sort_index()
    churn_pct = df['Churn'].value_counts(normalize=True).sort_index() * 100

    print("Distribución de Churn:")
    if 0 in churn_counts.index:
        print(f"• Clientes que NO se fueron (Churn=0): {churn_counts[0]:,} ({churn_pct[0]:.1f}%)")
    if 1 in churn_counts.index:
        print(f"• Clientes que SÍ se fueron (Churn=1): {churn_counts[1]:,} ({churn_pct[1]:.1f}%)")

    # Tasa de churn
    tasa_churn = (df['Churn'] == 1).mean() * 100
    print(f"\n🚨 TASA DE CHURN: {tasa_churn:.1f}%")

    return churn_counts, churn_pct

# Ejemplo de uso
churn_counts, churn_pct = analisis_churn_basico(df_clean)

📊 ANÁLISIS DE CHURN (Variable Objetivo)
----------------------------------------
Distribución de Churn:
• Clientes que NO se fueron (Churn=0): 5,174 (73.5%)
• Clientes que SÍ se fueron (Churn=1): 1,869 (26.5%)

🚨 TASA DE CHURN: 26.5%


#📄Informe final

In [21]:
def analisis_demografico(df: pd.DataFrame):
    print("\n👥 ANÁLISIS DEMOGRÁFICO")
    print("-" * 40)

    def pct(ct, row_label):
        """Devuelve % churn (col=1) para la fila row_label, o NaN si no existe"""
        if 1 not in ct.columns or row_label not in ct.index:
            return np.nan
        return float(ct.loc[row_label, 1])

    # --- Género vs Churn ---
    if 'customer.gender' in df.columns:
        gender_churn = pd.crosstab(df['customer.gender'], df['Churn'], normalize='index') * 100
        print("Churn por Género:")
        print(f"• Hombres (1): {pct(gender_churn, 1):.1f}%")
        print(f"• Mujeres (0): {pct(gender_churn, 0):.1f}%")

    # --- Edad (SeniorCitizen) vs Churn ---
    if 'customer.SeniorCitizen' in df.columns:
        senior_churn = pd.crosstab(df['customer.SeniorCitizen'], df['Churn'], normalize='index') * 100
        print("\nChurn por Edad:")
        print(f"• Adultos mayores (1): {pct(senior_churn, 1):.1f}%")
        print(f"• Adultos jóvenes (0): {pct(senior_churn, 0):.1f}%")

    # --- Pareja vs Churn ---
    if 'customer.Partner' in df.columns:
        partner_churn = pd.crosstab(df['customer.Partner'], df['Churn'], normalize='index') * 100
        print("\nChurn por Estado Civil:")
        print(f"• Con pareja (1): {pct(partner_churn, 1):.1f}%")
        print(f"• Sin pareja (0): {pct(partner_churn, 0):.1f}%")
# Análisis demográfico
analisis_demografico(df_clean)


👥 ANÁLISIS DEMOGRÁFICO
----------------------------------------
Churn por Género:
• Hombres (1): 26.2%
• Mujeres (0): 26.9%

Churn por Edad:
• Adultos mayores (1): 41.7%
• Adultos jóvenes (0): 23.6%

Churn por Estado Civil:
• Con pareja (1): 19.7%
• Sin pareja (0): 33.0%


In [22]:

def analisis_servicios(df: pd.DataFrame):
    """
    Análisis de servicios contratados vs Churn.
    Revisa las variables de internet, teléfono y TV/películas
    para ver qué categorías presentan más churn.
    """
    print("\n📡 ANÁLISIS DE SERVICIOS")
    print("-" * 40)

    def mostrar_churn(col):
        """Imprime % de churn por categoría en la columna col"""
        if col not in df.columns:
            return
        ct = pd.crosstab(df[col], df['Churn'], normalize='index') * 100
        print(f"\nChurn según {col}:")
        for categoria in ct.index:
            churn_pct = ct.loc[categoria, 1] if 1 in ct.columns else np.nan
            print(f"• {categoria}: {churn_pct:.1f}% de churn")

    # --- Teléfono ---
    mostrar_churn('phone.MultipleLines')

    # --- Internet y servicios asociados ---
    mostrar_churn('internet.InternetService')
    mostrar_churn('internet.OnlineSecurity')
    mostrar_churn('internet.OnlineBackup')
    mostrar_churn('internet.DeviceProtection')
    mostrar_churn('internet.TechSupport')
    mostrar_churn('internet.StreamingTV')
    mostrar_churn('internet.StreamingMovies')

analisis_servicios(df_clean)


📡 ANÁLISIS DE SERVICIOS
----------------------------------------

Churn según phone.MultipleLines:
• No: 25.0% de churn
• No phone service: 24.9% de churn
• Yes: 28.6% de churn

Churn según internet.InternetService:
• DSL: 19.0% de churn
• Fiber optic: 41.9% de churn
• No: 7.4% de churn

Churn según internet.OnlineSecurity:
• No: 41.8% de churn
• No internet service: 7.4% de churn
• Yes: 14.6% de churn

Churn según internet.OnlineBackup:
• No: 39.9% de churn
• No internet service: 7.4% de churn
• Yes: 21.5% de churn

Churn según internet.DeviceProtection:
• No: 39.1% de churn
• No internet service: 7.4% de churn
• Yes: 22.5% de churn

Churn según internet.TechSupport:
• No: 41.6% de churn
• No internet service: 7.4% de churn
• Yes: 15.2% de churn

Churn según internet.StreamingTV:
• No: 33.5% de churn
• No internet service: 7.4% de churn
• Yes: 30.1% de churn

Churn según internet.StreamingMovies:
• No: 33.7% de churn
• No internet service: 7.4% de churn
• Yes: 29.9% de churn


In [23]:
def analisis_financiero(df: pd.DataFrame):
    """
    Análisis financiero: cargos y métodos de pago vs Churn.
    """
    print("\n💰 ANÁLISIS FINANCIERO")
    print("-" * 40)

    # --- Resumen de cargos mensuales y totales ---
    if 'account.Charges.Monthly' in df.columns and 'account.Charges.Total' in df.columns:
        print("📈 Resumen de cargos:")
        print(f"• Cargo mensual promedio: {df['account.Charges.Monthly'].mean():.2f}")
        print(f"• Cargo mensual mínimo: {df['account.Charges.Monthly'].min():.2f}")
        print(f"• Cargo mensual máximo: {df['account.Charges.Monthly'].max():.2f}")
        print(f"• Cargo total promedio: {df['account.Charges.Total'].mean():.2f}")
        print(f"• Cargo total mínimo: {df['account.Charges.Total'].min():.2f}")
        print(f"• Cargo total máximo: {df['account.Charges.Total'].max():.2f}")

        # Cargos por churn
        cargos_churn = df.groupby('Churn')[['account.Charges.Monthly','account.Charges.Total']].mean()
        print("\n💸 Promedio de cargos por Churn:")
        print(cargos_churn)

    # --- Métodos de pago ---
    if 'account.PaymentMethod' in df.columns:
        print("\n💳 Churn según Método de Pago:")
        ct = pd.crosstab(df['account.PaymentMethod'], df['Churn'], normalize='index') * 100
        for metodo in ct.index:
            churn_pct = ct.loc[metodo, 1] if 1 in ct.columns else np.nan
            print(f"• {metodo}: {churn_pct:.1f}% de churn")

    # --- Contrato ---
    if 'account.Contract' in df.columns:
        print("\n📃 Churn según Tipo de Contrato:")
        ct = pd.crosstab(df['account.Contract'], df['Churn'], normalize='index') * 100
        for contrato in ct.index:
            churn_pct = ct.loc[contrato, 1] if 1 in ct.columns else np.nan
            print(f"• {contrato}: {churn_pct:.1f}% de churn")

    # --- Facturación sin papel ---
    if 'account.PaperlessBilling' in df.columns:
        print("\n🧾 Churn según Facturación Electrónica:")
        ct = pd.crosstab(df['account.PaperlessBilling'], df['Churn'], normalize='index') * 100
        if 1 in ct.index:
            print(f"• Sí (1): {ct.loc[1, 1]:.1f}% de churn")
        if 0 in ct.index:
            print(f"• No (0): {ct.loc[0, 1]:.1f}% de churn")
            
# Análisis financiero
analisis_financiero(df_clean)



💰 ANÁLISIS FINANCIERO
----------------------------------------
📈 Resumen de cargos:
• Cargo mensual promedio: 64.72
• Cargo mensual mínimo: 18.25
• Cargo mensual máximo: 118.75
• Cargo total promedio: 2280.63
• Cargo total mínimo: 18.80
• Cargo total máximo: 8684.80

💸 Promedio de cargos por Churn:
       account.Charges.Monthly  account.Charges.Total
Churn                                                
0                    61.265124            2555.344141
1                    74.441332            1531.796094

💳 Churn según Método de Pago:
• Bank transfer (automatic): 16.7% de churn
• Credit card (automatic): 15.2% de churn
• Electronic check: 45.3% de churn
• Mailed check: 19.1% de churn

📃 Churn según Tipo de Contrato:
• Month-to-month: 42.7% de churn
• One year: 11.3% de churn
• Two year: 2.8% de churn

🧾 Churn según Facturación Electrónica:
• Sí (1): 33.6% de churn
• No (0): 16.3% de churn


In [24]:
def analisis_financiero(df: pd.DataFrame):
    """
    Análisis financiero: cargos y métodos de pago vs Churn.
    """
    print("\n💰 ANÁLISIS FINANCIERO")
    print("-" * 40)

    # --- Resumen de cargos mensuales y totales ---
    if 'account.Charges.Monthly' in df.columns and 'account.Charges.Total' in df.columns:
        print("📈 Resumen de cargos:")
        print(f"• Cargo mensual promedio: {df['account.Charges.Monthly'].mean():.2f}")
        print(f"• Cargo mensual mínimo: {df['account.Charges.Monthly'].min():.2f}")
        print(f"• Cargo mensual máximo: {df['account.Charges.Monthly'].max():.2f}")
        print(f"• Cargo total promedio: {df['account.Charges.Total'].mean():.2f}")
        print(f"• Cargo total mínimo: {df['account.Charges.Total'].min():.2f}")
        print(f"• Cargo total máximo: {df['account.Charges.Total'].max():.2f}")

        # Cargos por churn
        cargos_churn = df.groupby('Churn')[['account.Charges.Monthly','account.Charges.Total']].mean()
        print("\n💸 Promedio de cargos por Churn:")
        print(cargos_churn)

    # --- Métodos de pago ---
    if 'account.PaymentMethod' in df.columns:
        print("\n💳 Churn según Método de Pago:")
        ct = pd.crosstab(df['account.PaymentMethod'], df['Churn'], normalize='index') * 100
        for metodo in ct.index:
            churn_pct = ct.loc[metodo, 1] if 1 in ct.columns else np.nan
            print(f"• {metodo}: {churn_pct:.1f}% de churn")

    # --- Contrato ---
    if 'account.Contract' in df.columns:
        print("\n📃 Churn según Tipo de Contrato:")
        ct = pd.crosstab(df['account.Contract'], df['Churn'], normalize='index') * 100
        for contrato in ct.index:
            churn_pct = ct.loc[contrato, 1] if 1 in ct.columns else np.nan
            print(f"• {contrato}: {churn_pct:.1f}% de churn")

    # --- Facturación sin papel ---
    if 'account.PaperlessBilling' in df.columns:
        print("\n🧾 Churn según Facturación Electrónica:")
        ct = pd.crosstab(df['account.PaperlessBilling'], df['Churn'], normalize='index') * 100
        if 1 in ct.index:
            print(f"• Sí (1): {ct.loc[1, 1]:.1f}% de churn")
        if 0 in ct.index:
            print(f"• No (0): {ct.loc[0, 1]:.1f}% de churn")
# Análisis de contratos
analisis_financiero(df_clean)


💰 ANÁLISIS FINANCIERO
----------------------------------------
📈 Resumen de cargos:
• Cargo mensual promedio: 64.72
• Cargo mensual mínimo: 18.25
• Cargo mensual máximo: 118.75
• Cargo total promedio: 2280.63
• Cargo total mínimo: 18.80
• Cargo total máximo: 8684.80

💸 Promedio de cargos por Churn:
       account.Charges.Monthly  account.Charges.Total
Churn                                                
0                    61.265124            2555.344141
1                    74.441332            1531.796094

💳 Churn según Método de Pago:
• Bank transfer (automatic): 16.7% de churn
• Credit card (automatic): 15.2% de churn
• Electronic check: 45.3% de churn
• Mailed check: 19.1% de churn

📃 Churn según Tipo de Contrato:
• Month-to-month: 42.7% de churn
• One year: 11.3% de churn
• Two year: 2.8% de churn

🧾 Churn según Facturación Electrónica:
• Sí (1): 33.6% de churn
• No (0): 16.3% de churn


In [28]:
def generar_conclusiones(df: pd.DataFrame, df_original: pd.DataFrame = None):
    """
    Genera conclusiones narrativas de churn y calidad de datos
    usando el dataset limpio df. 
    (Opcional: pasa df_original para comparar tamaños).
    """
    print("🎯 HALLAZGOS PRINCIPALES")
    print("-" * 50)

    # --- Tasa global de churn ---
    tasa_churn = (df['Churn'] == 1).mean() * 100
    print(f"• Tasa de churn actual: {tasa_churn:.1f}%")

    # Factores de riesgo
    factores_riesgo = []
    recomendaciones = []

    # 1. Tipo de contrato
    if 'account.Contract' in df.columns:
        contract_churn = pd.crosstab(df['account.Contract'], df['Churn'], normalize='index')
        if 1 in contract_churn.columns:
            contract_churn = contract_churn[1] * 100
            max_churn_contract = contract_churn.idxmax()
            churn_val = contract_churn.max()
            factores_riesgo.append(
                f"Contratos '{max_churn_contract}' muestran mayor churn ({churn_val:.1f}%)"
            )
            recomendaciones.append(
                f"Fomentar migración de clientes con contrato '{max_churn_contract}' hacia contratos de mayor permanencia mediante descuentos o beneficios."
            )

    # 2. Duración (tenure)
    if 'customer.tenure' in df.columns:
        avg_churn = df[df['Churn']==1]['customer.tenure'].mean()
        avg_stay  = df[df['Churn']==0]['customer.tenure'].mean()
        if avg_churn < avg_stay:
            factores_riesgo.append(
                f"Clientes con menor antigüedad se van antes (promedio: {avg_churn:.1f} meses vs {avg_stay:.1f})"
            )
            recomendaciones.append(
                "Implementar programas de onboarding y fidelización temprana en los primeros 6 meses del cliente."
            )

    # 3. Cargos mensuales
    if 'account.Charges.Monthly' in df.columns:
        avg_monthly_churn = df[df['Churn']==1]['account.Charges.Monthly'].mean()
        avg_monthly_stay  = df[df['Churn']==0]['account.Charges.Monthly'].mean()
        if avg_monthly_churn > avg_monthly_stay:
            factores_riesgo.append(
                f"Clientes con cargos mensuales más altos tienden a irse (${avg_monthly_churn:.2f} vs ${avg_monthly_stay:.2f})"
            )
            recomendaciones.append(
                "Revisar estructura de precios o agregar valor adicional (ej. bundles, servicios premium incluidos) para clientes de alto cargo mensual."
            )

    # Mostrar factores
    print("\n⚠️ PRINCIPALES FACTORES DE RIESGO")
    if factores_riesgo:
        for i, f in enumerate(factores_riesgo, 1):
            print(f"{i}. {f}")
    else:
        print("No se detectaron factores de riesgo destacados.")

    # --- Recomendaciones ---
    print("\n💡 RECOMENDACIONES ESTRATÉGICAS")
    if recomendaciones:
        for i, r in enumerate(recomendaciones, 1):
            print(f"{i}. {r}")
    else:
        print("No se generaron recomendaciones específicas.")



# Ejemplo de uso
generar_conclusiones(df_clean, df)


🎯 HALLAZGOS PRINCIPALES
--------------------------------------------------
• Tasa de churn actual: 26.5%

⚠️ PRINCIPALES FACTORES DE RIESGO
1. Contratos 'Month-to-month' muestran mayor churn (42.7%)
2. Clientes con menor antigüedad se van antes (promedio: 18.0 meses vs 37.6)
3. Clientes con cargos mensuales más altos tienden a irse ($74.44 vs $61.27)

💡 RECOMENDACIONES ESTRATÉGICAS
1. Fomentar migración de clientes con contrato 'Month-to-month' hacia contratos de mayor permanencia mediante descuentos o beneficios.
2. Implementar programas de onboarding y fidelización temprana en los primeros 6 meses del cliente.
3. Revisar estructura de precios o agregar valor adicional (ej. bundles, servicios premium incluidos) para clientes de alto cargo mensual.


In [29]:
df_clean.to_csv("TelecomX_LATAM_clean.csv", index=False)
print("✅ Archivo guardado como TelecomX_LATAM_clean.csv")

✅ Archivo guardado como TelecomX_LATAM_clean.csv
